##### EDA

In [2]:
import torchxrayvision as xrv

C:\Program Files\Python312\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
d = xrv.datasets.COVID19_Dataset(imgpath="data/images/", csvpath="data/csv/metadata.csv")

In [4]:
img = d[0]['img'][0]
print(f"sample image of shape {img.shape}")
print(f"dataset of length {len(d)}")
print(img)

sample image of shape (888, 882)
dataset of length 535
[[1024.      1015.9686  1007.93726 ... -469.83527 -477.86664 -477.86664]
 [1024.      1024.      1015.9686  ... -493.92938 -493.92938 -493.92938]
 [1024.      1024.      1015.9686  ... -501.96075 -509.99213 -509.99213]
 ...
 [1024.      1024.       983.84314 ... -172.6745  -156.61176 -148.58038]
 [1024.      1024.       983.84314 ... -252.98822 -244.95685 -244.95685]
 [1024.      1024.       983.84314 ... -437.70978 -429.6784  -421.64703]]


##### Dataloader

In [1]:
from dataLoader import *
from torch.utils.data import DataLoader

C:\Users\Hoo Kai Sng\OneDrive - National University of Singapore\Documents\NUS\CS3244\Project\xrayenv\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# train_dataset = Covid19DataSet('train', transform = 'vanilla')
augmented_dataset = Covid19DataSet('train', transform = 'augment')
validation_dataset = Covid19DataSet('val', transform = 'vanilla')
test_dataset = Covid19DataSet('test', transform = 'vanilla')

In [3]:
BATCH_SIZE = 16
train_loader = DataLoader(dataset = augmented_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
validation_loader = DataLoader(dataset = validation_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)
test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)

In [4]:
# from PIL import Image
# import torchvision.transforms as transforms
# d = xrv.datasets.COVID19_Dataset(imgpath = "data/images/", csvpath = "data/csv/metadata.csv")
# img = d[0]['img'][0]
# img = img + 1024
# img = np.divide(img, 2048)
# pil = Image.fromarray(img)
# ld = transforms.ToTensor()
# tensor = ld(pil)
# compose = transforms.Compose([
#     transforms.Resize((224,224)),
#     transforms.RandomHorizontalFlip(p = 1),
#     transforms.ToTensor(), 
# ])
# tensor2 = compose(pil)
# print(tensor2)

##### Building of model

In [5]:
from models import *
from torchvision import models
from torchinfo import summary  # or use torchsummary

In [6]:
ResNet18_Model = models.resnet18(weights = 'DEFAULT')
# summary(ResNet18_Model, input_size=(16, 3, 224, 224), depth = 3)
# print(ResNet18_Model)

In [7]:
mini_model = ConvNetGlobPooling()
# summary(model_22, (16, 1, 224, 224), depth = 3)

##### Training Model

In [8]:
from evaluation import *

In [9]:
globpool_model = train_model(mini_model, train_loader, validation_loader)

Training model...
Epoch [1/5] |Training Loss: 0.6183 | Validation Loss: 2.7044
Epoch [2/5] |Training Loss: 0.5661 | Validation Loss: 2.5211
Epoch [3/5] |Training Loss: 0.5703 | Validation Loss: 2.5313
Epoch [4/5] |Training Loss: 0.5668 | Validation Loss: 2.5193
Epoch [5/5] |Training Loss: 0.6260 | Validation Loss: 2.7168
Training completed in 10mins 34s
Best Loss: 2.5193017224470773


In [10]:
resnet_model = retrain_model(ResNet18_Model, train_loader, validation_loader)

Training model...
Epoch [1/5] |Training Loss: 4.7753 | Validation Loss: 17.8786
Epoch [2/5] |Training Loss: 1.7805 | Validation Loss: 9.7228
Epoch [3/5] |Training Loss: 3.0484 | Validation Loss: 12.6606
Epoch [4/5] |Training Loss: 1.4876 | Validation Loss: 5.5806
Epoch [5/5] |Training Loss: 1.2749 | Validation Loss: 8.4158
Training completed in 11mins 38s
Best Loss: 5.580577251975835


##### Evaluation

In [13]:
mm_prec, t, f, n = evaluate_model(globpool_model, test_loader, 'prec') 
mm_rec, t1, f1, n1 = evaluate_model(globpool_model, test_loader, 'rec')
print(t, f, n)
print(f"Mini Model: Precision = {mm_prec:.0f}% | Recall = {mm_rec:.0f}%")

TypeError: 'collections.OrderedDict' object is not callable

In [ ]:
resnet_prec = evaluate_model(resnet model, test_loader, 'prec') 
resnet_rec = evaluate_model(resnet_model, test_loader, 'rec')
print(f"Resnet18 Model: Precision = {resnet_prec:.0f}% | Recall = {resnet_rec:.0f}%")

##### Visualisation